# Constructing a Covariance Matrix

Let's make it simple and say we have 4 distances and 2 methods, so both methods 
have two SNe each. Let's say the distances are `D1, D2, D3`, and `D4`, while the 
errors are `eD1, eD2, eD3`, and `eD4`. Let's further say that `D1` and `D2` are 
Cepheid distances and `D3` and `D4` are TRGB. And lastly, the "extra" error 
for methods is `sd_cal`. Then we have the distance vector   `dist = [D1, D2, D3, D4]` and the error   `edist = [eD1, eD2, eD3, eD4]`. We also must have a vector of methods, say   `methods = ['c', 'c', 't', 't']`:


In [5]:
import numpy as np

# Just some random numbers to do the example
dist = np.array([34.,35., 32., 31.])
edist = np.array([0.6, 0.3, 0.5, 0.9])
sd_cal = 0.03
methods = np.array(['c','c','t','t'])

Easiest thing to do is start with a covariance matrix that has the variances as the diagonal. After a google search, `np.fill_diagonal` seems to be the right thing to use.
  


In [9]:
covmat_diag = np.zeros((len(dist),len(dist)))
np.fill_diagonal(covmat_diag, edist**2)
print(covmat)

[[0.3609 0.0009 0.     0.    ]
 [0.0009 0.0909 0.     0.    ]
 [0.     0.     0.2509 0.0009]
 [0.     0.     0.0009 0.8109]]


Now we just need the systematic terms, which will be sd_cal**2 whenever methods are the same. You could do this with a for-loop, but we want to be efficient, so we want a way to do it with numpy calls. There is a sneaky way to do this involving "newaxis". There's a nice explanation here:  https://www.gormanalysis.com/blog/python-numpy-for-your-grandma-3-2-newaxis/

In this case we're not doing subtraction, but element comparison, and we're doing it on two "copies" of the same array. It takes some time to get your brain around this, but it's very very handy when trying to vectorize code. What we can do is construct a boolean (True/False) 4X4 array `method_same[i,j]` where the value is True if `methods[i] == methods[j]`, and False otherwise. Here's how to do it:

In [10]:
method_same = (methods[:,np.newaxis] == methods[np.newaxis,:])
print(method_same)

[[ True  True False False]
 [ True  True False False]
 [False False  True  True]
 [False False  True  True]]


And now, because we can use the fact that True is 1 and False is 0 when you do arithmatic on boolean arrays, we just add this matrix times `sd_cal**2` to our covariance matrix.

In [11]:
covmat = covmat_diag + method_same*sd_cal**2
print(covmat)

[[0.3609 0.0009 0.     0.    ]
 [0.0009 0.0909 0.     0.    ]
 [0.     0.     0.2509 0.0009]
 [0.     0.     0.0009 0.8109]]


In fact, since `covmat_diag` and `method_same` never change, you can compute them ahead of time (outside the `lnlike` function) and just sum up the parts at each MCMC iteration. Now you should be able to take this into your code. All these "tricks" should work on `astropy.table` columns as well as `np.array`s.